In [1]:
'''
https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system
'''
import pandas as pd 
import numpy as np 
#Import Tf*IdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
import json

In [ ]:
{
        "model": "movies.movie",
        "pk": 49520,
        "fields": {
            "adult": false,
            "backdrop_path": "/z5xeSCrTLpA4VSLtbwakWWEXWM0.jpg",
            "genre_ids": [
                35
            ],
            "original_language": "en",
            "original_title": "The Change-Up",
            "overview": "Dave is a married man with two kids and a loving wife, and Mitch is a single man who is at the prime of his sexual life. One fateful night while Mitch and Dave are peeing in a fountain when lightning strikes, they switch bodies.",
            "popularity": 54.351,
            "poster_path": "/j1XdODJMwGfR4A4AlgsDpBhhLx0.jpg",
            "release_date": "2011-08-05",
            "title": "The Change-Up",
            "vote_average": 6.1,
            "vote_count": 2145,
            "like_users": [],
            "movie_reference_genre": [],
            "movie_reference_overview": []
        }
    },

In [2]:
#나중에 루트 수정시 활용
json_path="tmdb.json"

#contents에 json파일들 복사,이식
with open(json_path,'r') as j:
    contents=json.loads(j.read())

#print(contents)
#데이터 프레임 짜서 집어넣을것
#칼럼명은 model, pk, adult, backdrop_path, genre_ids, original_language, original_title, overview, popularity, poster_path, release_date,
#title, vote_average, vote_count, like_users, movie_reference_genre, movie_reference_overview

[{'model': 'movies.genre', 'pk': 28, 'fields': {'name': 'Action'}}, {'model': 'movies.genre', 'pk': 12, 'fields': {'name': 'Adventure'}}, {'model': 'movies.genre', 'pk': 16, 'fields': {'name': 'Animation'}}, {'model': 'movies.genre', 'pk': 35, 'fields': {'name': 'Comedy'}}, {'model': 'movies.genre', 'pk': 80, 'fields': {'name': 'Crime'}}, {'model': 'movies.genre', 'pk': 99, 'fields': {'name': 'Documentary'}}, {'model': 'movies.genre', 'pk': 18, 'fields': {'name': 'Drama'}}, {'model': 'movies.genre', 'pk': 10751, 'fields': {'name': 'Family'}}, {'model': 'movies.genre', 'pk': 14, 'fields': {'name': 'Fantasy'}}, {'model': 'movies.genre', 'pk': 36, 'fields': {'name': 'History'}}, {'model': 'movies.genre', 'pk': 27, 'fields': {'name': 'Horror'}}, {'model': 'movies.genre', 'pk': 10402, 'fields': {'name': 'Music'}}, {'model': 'movies.genre', 'pk': 9648, 'fields': {'name': 'Mystery'}}, {'model': 'movies.genre', 'pk': 10749, 'fields': {'name': 'Romance'}}, {'model': 'movies.genre', 'pk': 878, '

In [ ]:
'''
https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system
'''

df2=pd.read_csv('Movies.csv', encoding='utf-8')

tfidf = TfidfVectorizer(stop_words='english')  # the나 a 같은 단어(이런걸 stop_word라고 한다.) 지우기 
df2['overview'] = df2['overview'].fillna('')  # overview 없으면 빈문자열로 채우고
tfidf_matrix = tfidf.fit_transform(df2['overview'])  #Construct the required TF-IDF matrix by fitting and transforming the data

# print(tfidf_matrix.shape)  # 4803개 영화 중, 20978개의 언어
# 우리가 채택한 방식은 코사인 유사성(cosine similarity score), 크기가 무관하고 계산이 빠름, sklearn의 linear_kernel()이 빠른 계산 지원
from sklearn.metrics.pairwise import linear_kernel
# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 제목을 입력 받아 가장 유사한 10개의 영화목록을 출력하는 함수를 정의
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates() # 영화 이름 입력시 인덱스를 식별하는 메커니즘이 필요하므로 영화 제목과 DataFrame 인덱스간의 역매핑

def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]  # 이름으로 인덱스 긁어오기
    sim_scores = list(enumerate(cosine_sim[idx]))  # 그 영화와 관련된 문자열 코사인 유사성 다 긁어오기
    try:
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)  # 코사인 유사성 순으로 정렬
    except:
        print(title)
    sim_scores = sim_scores[1:11]  # 상위 10개 슬라이스
    movie_indices = [i[0] for i in sim_scores]  # 그 상위 10개의 인덱스를 매핑
    if len(movie_indices) < 10:
        movie_indices = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10] 
    # return movie_indices  # id 결과 반환
    return df2['title'].iloc[movie_indices]  # Object 결과 반환


# print(get_recommendations('The Dark Knight Rises'))
# get_recommendations('The Avengers').column

# 반복문으로 df 채우기
# answers = pd.DataFrame(columns=['title', 'recommended'])
# idx = 0
# for d in df2['title']:
#     answers.loc[idx] = [d, get_recommendations(d)]
#     idx += 1
# answers.to_csv('recommended_movie_detail.csv', index=False, encoding='utf-8')


# 반복문으로 df 채우기
answers = pd.DataFrame(columns=['title', 
                                'recommended1',
                                'recommended2',
                                'recommended3',
                                'recommended4',
                                'recommended5',
                                'recommended6',
                                'recommended7',
                                'recommended8',
                                'recommended9',
                                'recommended10'])
idx = 0
for d in df2['title']:
    answers_list = [d] + list(get_recommendations(d).values)
    if len(answers_list) == 11:
        answers.loc[idx] = answers_list
        idx += 1
    else:
        print('what?!')
answers.to_csv('recommended_movie_detail.csv', index=False, encoding='utf-8')
